## Download das imagens do BDC

Utiliza os scripts em scr/data/BDC_downloader.py


In [ ]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))
from rasterio.coords import BoundingBox
import rasterio

import src.data.BDC_downloader as BDC_downloader


### Define a região do tile para o download


- É necessário definir qual tile será baixado.
- É feita a busca no BDC explorer 
- Deve-se adicionar a latitude e longitude aproximada para diminuir o tempo de busca




In [ ]:

tiles = {'MG':'032027',
         'RS': '025037',
         'AM':'016009',
         'BA':'038019',
         'DF': '028022',
         'RJ': '033029'}


In [ ]:

list(tiles.values())


Obtém a URI dos items do tile.

Não esqueça de colocar a bounding_box

In [ ]:
import pystac_client


items = BDC_downloader.get_tiles(tiles=list(tiles.values()),
              collections=['S2-16D-2'], 
              datetime='2019-01-01/2019-12-31'
              )


In [ ]:
items[0]

In [ ]:
it = items[0].to_dict()
it['assets']['SCL']

In [ ]:
it['assets']['SCL']['href']


In [ ]:
#items[0]['properties']
it = items[0].to_dict()
it['properties']['datetime']

In [ ]:
len(items)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from datetime import datetime

def plot_valid_coverage(by = 'date'):
    ### by='ordered'
    ### by='reversed'
    working_dir = os.path.abspath('..')

    for uf, tile in tiles.items():
        print(uf, tile)

        # Store results
        

        tile_items = []
        for it in items:
            it = it.to_dict()
            if it['properties']['bdc:tiles'][0] == tile:  # Tile filtering
                tile_items.append(it)

        print()
        counts = []
        if by != 'time':
            for i, it in enumerate(tile_items):
                uri = it['assets']['SCL']['href']
                with rasterio.open(uri) as src:
                    data = src.read()
                arr = np.where((data >= 4) & (data <= 6), 1, 0)  # Binary mask (valid coverage)
                valid_pixel_count = np.sum(arr) / arr.size  # Count number of 1s in arr
                counts.append(valid_pixel_count)  # Store it
            
            ordered_idx = np.argsort(counts)
            if by == 'reversed':
                ordered_idx = ordered_idx[::-1]
                
        # **Sort tile_items by date**
        if by == 'date':
            tile_items.sort(key=lambda it: datetime.fromisoformat(it['properties']['datetime'][:10]))
        else:
            tile_items = [tile_items[i] for i in ordered_idx]
        #tile_items.sort(key=lambda it: datetime.fromisoformat(it['properties']['datetime'][:10]))
        
        # ---------------------------------
        selected_arrays = []
        sum_overlap = None  # Track total covered area
        coverage_metric = []  # Store how coverage improves with more arrays
        counts = []  # Store the number of valid pixels in each arr
        dates = []  # Store corresponding dates
        for i, it in enumerate(tile_items):
            uri = it['assets']['SCL']['href']
            date = datetime.fromisoformat(it['properties']['datetime'][:10]).strftime('%Y-%m-%d')  # Format date
            dates.append(date)

            # Read raster
            with rasterio.open(uri) as src:
                data = src.read()
            arr = np.where((data >= 4) & (data <= 6), 1, 0)  # Binary mask (valid coverage)
            valid_pixel_count = np.sum(arr) / arr.size  # Count number of 1s in arr
            counts.append(valid_pixel_count)  # Store it

            if sum_overlap is None:
                sum_overlap = np.zeros_like(arr, dtype=bool)  # Initialize sum array

            # Compute new overlap if this array is added
            new_overlap = np.sum(np.logical_or(sum_overlap, arr))
            
            # Improvement calculation
            improvement = new_overlap if len(selected_arrays) == 0 else new_overlap - np.sum(sum_overlap)
            coverage_metric.append(new_overlap / arr.size)  # Normalize by total pixels

            # Track selected images
            selected_arrays.append(arr)
            sum_overlap = np.logical_or(sum_overlap, arr)  # Update cumulative mask

        # **Plot coverage metric and valid pixel counts together**
        fig, ax1 = plt.subplots(figsize=(10, 5))

        
        ax1.set_ylim(0, 100)  # **Zoom in on y-axis for coverage metric**
        ax1.bar(dates, [100*c for c in counts], color="r", label="Pixels Válidos (%)")#, marker="s", linestyle="dashed", label="Pixels Válidos")
        ax1.set_ylabel("Percentual de Pixels Válidos (%)", color="r")
        ax1.tick_params(axis="y", labelcolor="r")
        ax1.set_xticks(dates)  # Use dates as x-axis labels
        ax1.set_xticklabels(dates, rotation=45, ha="right")  # Rotate labels for readability
        
        ax2 = ax1.twinx()  # Create a second y-axis
        ax2.plot(dates, [100*c for c in coverage_metric], marker="o", color="b", label="Cobertura (%)")
        ax2.set_xlabel("Datas das imagens")
        ax2.set_ylabel("Percentual de Cobertura Acumulada (%)", color="b")
        ax2.tick_params(axis="y", labelcolor="b")
        ax2.set_ylim(0, 100)  # **Zoom in on y-axis for coverage metric**
        ax2.grid(True, linestyle="--", alpha=0.7)  # Add grid
        ax2.grid(True, which='minor', color='0.5', linestyle='-')

        

        # **Add legend inside the plot (bottom right)**
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="lower right", frameon=True)

        fig.suptitle(f"Melhoria da Cobertura vs. Pixels Válidos ({uf}:{tile})")
        fig.tight_layout()

        save_to = os.path.join(working_dir, 'figs', f'Coverage_Counts_{by}_{uf}_{tile}.png')
        plt.savefig(save_to, bbox_inches="tight", pad_inches=0)
        plt.show()


In [ ]:
plot_valid_coverage(by = 'date')


In [ ]:
plot_valid_coverage(by = 'ordered')



In [ ]:
plot_valid_coverage(by = 'reversed')


### Download

- Faz o download e salva imagens tif no disco.
- Calcula automaticamente quantas datas vai baixar, de forma a maximizar a cobertura de pixels válidos, com menor número de datas.
- Será salvo em data/raw/S2-16D_V2_{tile}


In [ ]:

tiles = {'MG':'032027',
         'RS': '025037',
         'AM':'016009',
         'BA':'038019',
         'DF': '028022',
         'RJ': '033029'}

working_dir = os.path.abspath('..')
save_dir = os.path.join(working_dir,'data/raw')

for tile in tiles.values(): 
    items  = BDC_downloader.get_max_coverage_items(tile, N = 4, threshold = 98, collections=['S2-16D-2'], datetime='2019-01-01/2019-12-31')
    for item in items:
        BDC_downloader.download_and_save_item(item, save_dir = save_dir)



In [ ]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))
from rasterio.coords import BoundingBox
import rasterio

import src.data.BDC_downloader as BDC_downloader


In [ ]:

tiles_escolhidos = {
              'Boa Vista': '015002',  
              'Manaus': '016009',
              'Campo Grande': '021027',
              'Macapá': '025005',
              'Porto Alegre': '025037',
              'Curitiba': '027032',
              'Brasília': '028022',                      
              'Belo Horizonte': '032027',
              'Rio de Janeiro': '033029',
              'Teresina': '034011',
              'Petrolina': '036016',
              'Salvador': '038019',      
              }


In [ ]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))
from rasterio.coords import BoundingBox
import rasterio

import src.data.BDC_downloader as BDC_downloader


working_dir = os.path.abspath('..')
save_dir = os.path.join(working_dir,'data/raw')

#for tile in tiles_escolhidos.values(): 
for tile in ['016009']:
    items  = BDC_downloader.get_max_coverage_items(tile, N = 4, threshold = 98, collections=['S2-16D-2'], datetime='2019-01-01/2019-12-31')
    for item in items:
        BDC_downloader.download_and_save_item(item, save_dir = save_dir)